# USO DE LA LIBRERÍA LIBMF CON PYTHON

## OBJETIVO 

El objetivo de este notebook es mostrar la aplicación de algunas funciones/ métodos de la librería libmf con python, por lo que primero se importarán los siguientes paquetes. En caso de que no se tenga alguno instalado usar el comando `pip install`.

In [1]:
import sys
import numpy as np
import types
import scipy
from scipy.sparse import random
from scipy import stats
import pandas as pd
import copy
import random
from functools import partial
from colorama import init

import libmf
from libmf import mf

Using file found in /opt/anaconda3/lib/python3.7/site-packages:
/opt/anaconda3/lib/python3.7/site-packages/libmf.cpython-37m-darwin.so


A continuación se crean algunas funciones auxiliares que contienen su respectiva documentación.

### Funciones auxiliares para generar matrices ralas (con muchas entradas nulas o ceros)

In [2]:
def perform(fun, *args):
    """
    Evalua una función con los argumentos indicados
    
    params: fun         Una función
            *args       Parámetros que utiliza la función "fun"
                        (se pueden meter cuantos parámetros tenga 
                        dicha funcion)
    return: fun(*args)  Valor de fun con parámetros *args
    """
    return fun(*args)

In [3]:
def nan_or_value(x, f, *args):
    """
    Devuelve el valor de una función evaluada con sus parámetros
    *args acorde a la variable indicadora x
    
    params: x         boolean, si es 1 tómese valor de la función
                      si no será nan
            f         Una función
            *args     Parámetros que utiliza la función "f"
                      (se pueden meter cuantos parámetros tenga 
                      dicha funcion)
            
    return: 
        perform(f,*args) Si el valor de x es 1 y en otro caso regresa nan
    """
    if(x==1):
        return perform(f,*args)
    else:
        return np.nan

In [4]:
#Con esta vectorización se podrá aplicar a todos los elementos de un np.array la función nan_or_value
nan_or_value_vec=np.vectorize(nan_or_value, otypes=[object])

La siguiente función es la que crea matrices aletorias ralas. Es importante señalar, que por la forma de construcción pudiera no ser eficiente para matrices de dimensiones grandes.

In [5]:
def crear_matriz_rala(n_ren,m_col,prop_elem_difer_nan,func_rand,*args_func_rand):
    """
    Construye matriz rala de n_renxm_col, seleccionando al azar los elementos
    que serán nan cumpliendo que la proporción de éstos es de 
    1-prop_elem_difer_nan, donde los numeros diferentes de nan serán aleatorios
    generados a partir de una función aleatoria que se envíe como parámetro
    (y sus argumentos)
    
    params: n_ren               int número de renglones que tiene la matriz
            m_col               int número de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
            func_rand           Función que genera un sólo número aleatorio de
                                tipo float (no arreglos)
            *args_fun_rand      Argumentos necesarios para ejecutar la
                                función fun_rand
            
    return:  M                  np.ndarray que contiene matriz rala
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    #se juntan 2 arreglos, uno de ceros y otro de unos, para después
    #reacomodarlos al azar
    M=np.concatenate((np.zeros(tot-k).astype(int), np.ones(k).astype(int)))
    M=np.array(random.sample(list(M),tot))
    M=M.reshape(n_ren,m_col)
    #Los unos se intercambias por numeros aleatorios y los ceros
    #por nan (o se dejan como cero)
    M=nan_or_value_vec(M,func_rand,*args_func_rand).astype(float)
    return M

### Función que compacta las matrices ralas (quita las entradas que sean na o ceros)

In [6]:
def compactar_matriz_rala(M):
    """
    Pasa una matriz rala a su expresión compacta
    
    params: M  np.ndarray que representa matriz rala
    return: MC np.ndarray matriz en expresión compacta
               (no se reportarán los elementos con valor
               nan)
    """
    if type(M) is not np.ndarray:
        sys.exit('M debe ser de tipo numpy.ndarray')
    elif np.isnan(M).sum()==len(M):
        sys.exit('Todos los elementos de M son nan')
    MC=np.zeros((M[np.isnan(M)==False].size,3))
    k = 0
    n,m=M.shape
    for i in range(n): 
        for j in range(m): 
            if (~np.isnan(M[i,j])): 
                MC[k,0] = i
                MC[k,1] = j
                MC[k,2] = M[i,j] 
                k += 1
    return MC

### Función que crea una matriz aleatoria en el formato que lo requiere la librería LIBMF.

Este formato debe tener en la primer columna el índice de la fila (se podría pensar como el id del usuario), en la segunda el índice de la columna (se podría pensar como el id de la película) y finalmente el valor/calificación.

In [7]:
def crear_matriz_rala_compacta(n_ren, m_col, prop_elem_difer_nan):
    """
    Construye matriz de n_renxm_col, seleccionando al azar los elementos
    que serán distintos de nan. Éstos elementos distintos de nan serán
    números aleatorios entre 1 y 5 (uniforme discreta). Se dice compacta porque
    en esta matriz solo señala las coordenadas o entradas de la matriz que 
    son diferentes a nan
    
    params: n_ren               int número de renglones que tiene la matriz
            m_col               int número de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
    return: M  np.ndarray que contiene la matriz 
               (1era columna representa el índice de renglones,
               2da columna representa el índice de columnas,
               3era columna representa el valor del elemento
               o calificación en el contexto de netflix)
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=random.sample(range(tot), k)
    columnas=np.mod(M,m_col)
    renglones=np.divide(M,m_col).astype(int)
    aleatorios=np.random.randint(1, 6, k)
    
    M=np.vstack((renglones,columnas,aleatorios)).transpose()


    return M

### Función para generar las muestras de entrenamiento y validación en el formato que lo requiere la librería LIBMF

In [8]:
def muestra_val_ent(df,prop):
    """
    Separa al azar una base dada en 2 bases acorde una proporción de elementos
    dada
    
    params: df    pd.DataFrame (con 3 columnas=['usuario_id','peli_id',
                  'calif_x'])
            prop  float (0 < prop < 1) proporción de id_usuarios y
                  proporción de id_peliculas que se seleccionarán
    
    return: data_test      pd.DataFrame que tiene la base que se usará
                           para prueba (por construcción el número de
                           elementos de esta base no necesariamente coincide
                           con prop*total de elementos de df)
            data_training  pd.DataFrame que tiene la base que se usará
                           para entrenamiento
    """
    #Base de todos los usuarios
    usuarios=pd.DataFrame(df['usuario_id'].unique(), columns=['usuario_id'])
    #Muestreo de usuarios
    valida_usuarios=usuarios.sample(frac=prop)  

    #Base de todas las películas
    peliculas=pd.DataFrame(df['peli_id'].unique(), columns=['peli_id'])
    #Muestreo de películas
    valida_pelis=peliculas.sample(frac=prop)
    
    #df_valida tiene las columnas usuario_id, peli_id y calif, pero
    #sólo incluye los usuarios muestreados en valida_usuarios
    df_valida=pd.merge(df,valida_usuarios, on ='usuario_id', how='inner')
    #data_valida filtra de la base df_valida los pelis_id muestreas
    #en valida_pelis
    data_test=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
    left=pd.merge(df, data_test, how='left',on=['usuario_id', 'peli_id'])
    data_training=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]
    return data_test, data_training

### Función que imprime el valor de los párametros que se usaron para realizar el entrenamiento.

In [9]:
def imprime_atributos_MF(Obj):
    """
    Imprime los parámetros utilizados para factorización de la matriz contenida
    en Obj, mismos que son atributos del atributo _options
    
    params: Obj  libmf.mf.MF objeto de la librería libmf-python que contiene
                 la información requerida para hacer la factorización deseada
    
    return: None No regresa nada, sólo imprime los parámetros utilizados para
                 hacer la factorización
    """
    if type(Obj) is not libmf.mf.MF:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Atributos del modelo:')
    print('fun={0:d}'.format(Obj._options.fun))
    print('k={0:d}'.format(Obj._options.k))
    print('nr_threads={0:d}'.format(Obj._options.nr_threads))
    print('nr_bins={0:d}'.format(Obj._options.nr_bins))
    print('nr_iters={0:d}'.format(Obj._options.nr_iters))
    print('lambda_p1={0:0.4f}'.format(Obj._options.lambda_p1))
    print('lambda_p2={0:0.4f}'.format(Obj._options.lambda_p2))
    print('lambda_q1={0:0.4f}'.format(Obj._options.lambda_q1))
    print('lambda_q2={0:0.4f}'.format(Obj._options.lambda_q2))
    print('eta={0:0.4f}'.format(Obj._options.eta))
    print('do_nmf=',Obj._options.do_nmf)
    print('quiet=',Obj._options.quiet)
    print('copy_data=',Obj._options.copy_data)
    return None

### Función que imprime los datos del modelo entrenado

In [10]:
def imprime_atributos_model(Obj):
    """
    Imprime los atributos del objeto model (definido dentro dentro de la
    librería libmf)
    
    params: Obj  libmf.mf.MFModel objeto de la librería libmf-python que
                 contiene los resultados obtenidos de la corrida del
                 modelo de factorización
                 
    return: None No regresa nada, sólo imprime los resultados contenidos
            en el objeto model, generado con los métodos del objeto MF
    """
    if type(Obj) is not libmf.mf.MFModel:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Resultados model:')
    print('fun={0:d}'.format(Obj.fun))
    print('m={0:d}'.format(Obj.m))
    print('n={0:d}'.format(Obj.n))
    print('k={0:d}'.format(Obj.m))
    print('b={0:0.4f}'.format(Obj.b))
    print('P={0:0.4f}'.format(Obj.P.contents.value))
    print('Q={0:0.4f}'.format(Obj.Q.contents.value))
    return None

Con las funciones definidas anteriormente, se procede a hacer uso de ellas para probar las funciones/métodos de la librería LIBMF de python.

En primer lugar, se empezará por crear una matriz rala. Estas matrices son de nuestro interés ya que en el caso de las películas de Netflix, no todos los usuarios ven todas las películas o no todos califican las películas vistas, por lo que si se considera que el índice de las filas es el id de los usuarios y el índice de las columnas es el id de la película, entonces estas matrices contienen muchos "huecos". Por esta razón nos interesa generar ese tipo de matrices.

No obstante lo anterior, debido al numero de usuarios y películas, existen muchos datos, y para no guardar todos los valores de cero (o NA), se opta por expresar y guardar dicha matriz en una forma compacta, la cual contiene por cada calificación el usuario que la generó y la película a la que corrresponde (es decir, es una matriz con 3 columnas).

A continuación, se crea una matriz rala de 20 filas, 5 columnas, porcentaje de entradas diferentes de na del 25% (25 entradas diferentes de na) y con generación de enteros aleatorios entre el 1 y 5. 

No se omite señalar, que la generación de números enteros puede cambiarse por alguna otra función que se requiera, por ejemplo una constante o bernoulli.


In [11]:
M=crear_matriz_rala(20,5,0.25,stats.randint.rvs,*[1,6])

print('Matriz M:\n',M)
print('Tamaño M:\n',M.shape)
print('Numero de elementos diferentes de nan:\n',M[np.isnan(M)==False].size)

Matriz M:
 [[nan nan nan  3. nan]
 [nan  4. nan nan  1.]
 [ 3. nan  3. nan nan]
 [ 2. nan nan nan nan]
 [nan nan nan nan nan]
 [ 5. nan  1. nan nan]
 [nan nan  5. nan  2.]
 [nan nan nan  2. nan]
 [ 2. nan nan nan nan]
 [nan nan nan  3.  1.]
 [nan nan nan  2. nan]
 [nan  5. nan nan nan]
 [nan nan nan  1.  3.]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan  3.  3.  3. nan]
 [nan nan nan nan nan]
 [nan  4. nan nan nan]
 [ 4. nan  2. nan nan]
 [ 2. nan nan nan nan]]
Tamaño M:
 (20, 5)
Numero de elementos diferentes de nan:
 25


A continuación se compacta la matriz anterior, es decir, solo considerando las entradas donde se tenga un valor y 
donde la primer columna representa el índice de la fila, la segunda el índice de la columna y la tercera el valor.

**El formato descrito anteriormente es el que solicita la librería libmf.**

En este ejemplo se tendrá una matriz de 25 filas (las diferentes de na)

In [12]:
MC=compactar_matriz_rala(M)
print('Expresión compacta de M (MC):\n',MC)
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC):
 [[ 0.  3.  3.]
 [ 1.  1.  4.]
 [ 1.  4.  1.]
 [ 2.  0.  3.]
 [ 2.  2.  3.]
 [ 3.  0.  2.]
 [ 5.  0.  5.]
 [ 5.  2.  1.]
 [ 6.  2.  5.]
 [ 6.  4.  2.]
 [ 7.  3.  2.]
 [ 8.  0.  2.]
 [ 9.  3.  3.]
 [ 9.  4.  1.]
 [10.  3.  2.]
 [11.  1.  5.]
 [12.  3.  1.]
 [12.  4.  3.]
 [15.  1.  3.]
 [15.  2.  3.]
 [15.  3.  3.]
 [17.  1.  4.]
 [18.  0.  4.]
 [18.  2.  2.]
 [19.  0.  2.]]
Tamaño MC:
 (25, 3)


Dado que la librería requiere el formato descrito anteriormente, usaremos directamente la función "crea_matriz_rala_compacta" indicandole el número de filas, de columnas y el porcentaje de entradas distinto de na para que directamente devuelva una matriz compacta en el diseño que lo requiere la librería.

Dentro de las funciones de libmf existe una que se llama "generate_test_data" que genera las matrices en este formato, sin embargo, notamos que al usar ésta podría darse el caso de que una persona califique a una misma película con dos o más calificaciones diferentes, tal como se muestra en los registros marcados en rojo del siguiente ejemplo:

In [13]:
#El 1er parámetro indica que se seleccionará un valor
#de entre 0 y 19 para la 1er columna (que representa el índice
#de los renglones de la matriz rala que expresa); el 2do
#parámetro indica que se seleccionara al azar valores entre
#0 y 4 (que representa el índice de las columnas de la matriz
#rala que expresa); el 3er parámetro indica el número de elementos
#a generar 

np.random.seed(102)
MC=mf.generate_test_data(20, 5, 25)
print('Matriz generada con mf.generate_test_data(20,5,25):\n')
for i in range(MC.shape[0]):
    if MC[i,0]==0:
        print('\033[1;30;41m',MC[i,:])
    else:
        print('\033[0;0;0m',MC[i,:])
print('Tamaño MC:\n',MC.shape)

Matriz generada con mf.generate_test_data(20,5,25):

 [0.         1.         0.16637107]
 [19.          4.          0.30636304]
 [14.          3.          0.14721805]
 [18.          4.          0.22291025]
 [18.          1.          0.53951716]
 [15.          3.          0.74892651]
 [2.         2.         0.42831194]
 [11.          3.          0.10467911]
 [9.         3.         0.75337018]
 [8.         5.         0.59815912]
 [13.          0.          0.48677848]
 [8.         4.         0.97530286]
 [9.         2.         0.80230277]
 [15.          3.          0.33301479]
 [7.         5.         0.54855018]
 [4.         5.         0.31180599]
 [16.          4.          0.06230988]
 [6.         1.         0.90313856]
 [18.          0.          0.86054759]
 [3.        3.        0.6296915]
 [0.         1.         0.21655643]
 [0.        1.        0.5557169]
 [16.          5.          0.10932898]
 [6.         5.         0.55083566]
 [7.        0.        0.5812561]
Tamaño MC:
 (25, 3)


In [14]:
MC=crear_matriz_rala_compacta(20,5,0.25)
print('Expresión compacta de M (MC) ordenada por renglon:\n',MC[np.argsort(MC[:,0])])
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC) ordenada por renglon:
 [[ 0  0  2]
 [ 0  3  2]
 [ 2  3  4]
 [ 2  0  5]
 [ 3  1  2]
 [ 3  0  1]
 [ 4  3  5]
 [ 6  3  3]
 [ 8  0  1]
 [ 8  1  2]
 [10  2  5]
 [10  1  1]
 [11  3  3]
 [11  4  1]
 [13  2  4]
 [13  1  4]
 [13  0  3]
 [14  4  4]
 [15  0  5]
 [16  4  5]
 [16  3  4]
 [16  2  3]
 [17  4  3]
 [17  3  4]
 [19  3  1]]
Tamaño MC:
 (25, 3)


## CLASE MF DE LIBMF-PYTHON


A continuación se describe la clase base de la librería libmf en python que es la de **MF**. Con esta clase se realiza la factorización de la matriz R en otras dos matrices P y Q (entrenamiento del modelo) y se realizan las predicciones que se necesitan para tener un sistema de recomendación.

Se describe un resumen de su documentación:

* Clase: MF<br/>
Atributos:<br/>  
            .model
            ._options 
            .i
            .j
 Métodos:<br/>
          .fit -> < Objetivo: factorize the i x j data matrix X into (j, k) (k, i) 
                  <           sized matrices stored in MF.model
                  < Parámetro X: (n, 3) shaped numpy array [known index and values 
                  <              of the data matrix]

          .predict -> < Objetivo: assuming we have already run the fit method, predict
                      <           the values at certain indices of the data matrix
                      < Parámetro X: (n, 2) shaped numpy array
                      < Regresa: numpy array of length n

          .mf_cross_validation -> < Objetivo: Realizar cross-validation
                                  < Parámetro X: (n, 3)
                                  < Parámetro folds: number of train / test splits
                                  < Regresa: score across all folds 

          .q_factors -> < Objetivo: Obtener la matriz Q

          .p_factors -> < Objetivo:  Obtener la matriz P
          


En lo siguiente se muestran casos de su uso:

In [15]:
#objeto de tipo MF para obtener la factorización deseada
mf_netflix=mf.MF()

#veamos algunas de las propiedades del objeto mf_netflix
print('Tipo de objeto: ',type(mf_netflix))
print('Instance variables: ',mf_netflix.__dict__)

Tipo de objeto:  <class 'libmf.mf.MF'>
Instance variables:  {'model': None, '_options': <libmf.mf.MFParam object at 0x1a160a45f0>, 'i': None, 'j': None}


En la definición del objeto anterior no se especificó ningún parámetro, por lo que se usarán los valores de default de cada uno de ellos para entrenar el modelo (factorización de la matriz R).

Veamos cuáles son estos valores default.

In [16]:
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=8
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


Observemos también que esos parámetros se encuentran definidos dentro de los atributos del objeto:

In [17]:
print('Atributos: ', dir(mf_netflix._options))

Atributos:  ['__class__', '__ctypes_from_outparam__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_b_base_', '_b_needsfree_', '_fields_', '_objects', 'copy_data', 'do_nmf', 'eta', 'fun', 'k', 'lambda_p1', 'lambda_p2', 'lambda_q1', 'lambda_q2', 'nr_bins', 'nr_iters', 'nr_threads', 'quiet']


Sin embargo, en la definición del objeto se puede especificar el valor de cada parámetro para poder obtener la mejor factorización posible (sin llegar al sobreajuste)

In [18]:
mf_netflix=mf.MF(fun=0,k=10,nr_threads=12,nr_bins=26,nr_iters=20,lambda_p1=0.04,lambda_p2=0.0,lambda_q1=0.04,lambda_q2=0.0,eta=0.1,do_nmf=False,quiet=False,copy_data=True)
imprime_atributos_MF(mf_netflix)


Atributos del modelo:
fun=0
k=10
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


In [19]:
mf_netflix=mf.MF(eta=0.11,nr_iters=21, k=5)

imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=5
nr_threads=12
nr_bins=26
nr_iters=21
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1100
do_nmf= False
quiet= False
copy_data= True


### MÉTODO FIT DE LA CLASE MF

A este método se le tiene que pasar la matriz de entrenamiento (con el formato especificado anteriormente) y su objetivo es realizar la factorización de la matriz R, es decir, obtener P y Q.

Primero generemos la matriz de entrenamiento y validación.

In [20]:
ME=crear_matriz_rala_compacta(20,5,0.40)
MV=crear_matriz_rala_compacta(20,5,0.15)

Posteriormente, a estas dos muestras les daremos el siguiente tratamiento.

Es importante resaltar que solo vamos a considerar las primeras dos columnas (sin los valores/calificaciones) de "MV" dado que así lo requiere el método predict de esta clase para justo obtener las predicciones.

In [21]:
MV_index=MV[:,0:2]
MC_entrena=np.empty((40,3,))
MC_entrena[:]=np.nan
MC_valida=np.empty((15,2,))
MC_valida[:]=np.nan
for i in range(40):
    for j in range(3):
        MC_entrena[i,j]=float(ME[i,j])
        if i<15 and j<2:
            MC_valida[i,j]=float(MV_index[i,j])

Aplicamos el método fit  a la matriz de entrenamiento para obtener la factorización, en este ejemplo con los valores que se le asignan a los parámetros por default.

In [22]:
mf_netflix = mf.MF()
mf_netflix.fit(MC_entrena)

### MÉTODO VALIDACIÓN CRUZADA DE LA CLASE MF

A este método se le tiene que pasar la matriz de entrenamiento (con el formato especificado anteriormente) y el número de folds a realizar, en caso de que no se requiera el de default (5) y devuelve el score.


In [23]:
print('Validación cruzada con 5 folds: ',mf_netflix.mf_cross_validation(MC_entrena))

Validación cruzada con 5 folds:  2.3785476901224087


### MÉTODO PREDICT DE LA CLASE MF

A este método se le tiene que pasar la matriz de validación (con el formato especificado anteriormente) y devuelve un vector con las predicciones, con número de entradas igual al número de filas de la matriz de validación.


In [24]:
pred=mf_netflix.predict(MC_valida)
pred

array([2.4787939, 4.764296 , 5.5202403, 3.15     , 1.6136359, 3.15     ,
       3.443281 , 3.1817677, 1.3498484, 3.15     , 2.3174126, 3.15     ,
       2.0276985, 3.15     , 3.7285702], dtype=float32)

In [25]:
print('Los valores verdaderos de MC_valida son:',MV[:,2])

Los valores verdaderos de MC_valida son: [2 3 4 1 1 4 1 4 2 2 2 3 3 2 1]


### MÉTODO Q_FACTORS DE LA CLASE MF

Con este método se obtiene la información relacionada a la matriz Q

In [26]:
Q=mf_netflix.q_factors()
print('Tamaño Q:',Q.shape)
print('Matriz Q:\n',Q)

Tamaño Q: (5, 8)
Matriz Q:
 [[ 1.2824596   1.9227344  -0.08449509  1.3108523   0.4395189   0.72984236
   0.49773955  0.72080266]
 [ 1.1942284   0.37021667  0.79143095  0.5503865   1.3743138   0.6316633
   0.4063534   0.4624708 ]
 [ 0.6737016  -0.31932986  0.57967496  1.1403754   0.9068601   1.4687986
   0.90021425  0.2944856 ]
 [ 0.15856102  0.95147526  0.91506606  0.1824376   0.97885257 -0.01941632
   1.2071795   0.16623026]
 [ 0.8398119   0.5972035   0.97930104  1.0014008   1.1971345   0.7085809
   0.65009934  0.9450503 ]]


### MÉTODO P_FACTORS DE LA CLASE MF

Con este método se obtiene la información relacionada a la matriz P

In [27]:
P=mf_netflix.p_factors()
print('Tamaño P:',P.shape)
print('Matriz P:\n',P)

Tamaño P: (20, 8)
Matriz P:
 [[ 0.7206973   0.79033965 -0.04590722  0.9200396   0.16006885  0.7145977
   0.20753103  0.5761762 ]
 [ 0.90166247  0.37017277  0.65797573  0.44417265  0.92398804  0.3595448
   0.1421704   0.42120203]
 [ 0.33625796  0.95094436  0.69882685  0.37657622  0.7009829  -0.08166336
   0.62619096  0.43196923]
 [ 0.82073414 -0.06259158  0.3736268   0.9593312   0.73475814  1.0537355
   0.38770264  0.28462744]
 [ 0.58001685  0.3878115   0.71793765  0.8969061   1.0031044   0.58647907
   0.50661594  0.7210087 ]
 [        nan         nan         nan         nan         nan         nan
          nan         nan]
 [ 0.35646632  0.43328622  0.03154159  0.14622742  0.39964178  0.2189536
   0.1432136   0.11157191]
 [ 0.46941173  0.21514976  0.50287503  0.52952814  0.36221027  0.32730952
   0.1746976   0.6790201 ]
 [ 0.04431569  0.36034375  0.76247436  0.07356923  0.6243638   0.05653312
   0.9295621   0.0743401 ]
 [ 0.8118005   0.23356949  0.576844    0.8040227   0.8855678   0.8

En el siguiente ejemplo se indica que se realice la factorización no negativa, dado que se observa que en la matriz P anterior hay entradas negativas, que el número de iteraciones sea igual a 30 y que el número de factores latentes sea igual a 5.

In [28]:
mf_netflix = mf.MF(do_nmf=True, nr_iters=30, k=5)
mf_netflix.fit(MC_entrena)

In [29]:
pred=mf_netflix.predict(MC_valida)
pred

array([3.104402 , 4.9332056, 5.0038147, 3.15     , 1.588406 , 3.15     ,
       2.9180102, 3.1985285, 2.6134772, 3.15     , 1.3019586, 3.15     ,
       1.574866 , 3.15     , 4.2580276], dtype=float32)

In [30]:
print('Los valores verdaderos de MC_valida son:',MV[:,2])

Los valores verdaderos de MC_valida son: [2 3 4 1 1 4 1 4 2 2 2 3 3 2 1]


In [31]:
Q=mf_netflix.q_factors()
print('Tamaño Q:',Q.shape)
print('Matriz Q:\n',Q)

Tamaño Q: (5, 5)
Matriz Q:
 [[0.88836735 1.0538342  0.37675676 1.9469827  1.6698848 ]
 [0.9974352  0.21269749 0.00997171 1.7231356  1.0290359 ]
 [1.4064696  0.25710207 1.4692172  0.6137302  0.8299677 ]
 [0.         1.2025989  1.7132851  0.20057307 0.59910977]
 [0.2856683  1.0702928  1.2035244  1.310636   0.95362276]]


In [32]:
P=mf_netflix.p_factors()
print('Tamaño P:',P.shape)
print('Matriz P:\n',P)

Tamaño P: (20, 5)
Matriz P:
 [[6.7762518e-01 3.9102304e-01 9.7402394e-02 1.1923802e+00 9.3474168e-01]
 [7.0788229e-01 3.7968767e-01 0.0000000e+00 1.4646631e+00 9.2065686e-01]
 [0.0000000e+00 1.0929996e+00 8.3640170e-01 6.2606931e-01 5.4334170e-01]
 [1.3364741e+00 1.0597795e-01 5.2965903e-01 1.1503212e+00 8.8001317e-01]
 [4.6252418e-01 9.7816467e-01 9.8866612e-01 1.1759799e+00 1.0419403e+00]
 [          nan           nan           nan           nan           nan]
 [4.8608547e-01 3.8151646e-01 5.0716138e-01 2.8351045e-01 2.3018293e-01]
 [2.5578445e-01 5.2468550e-01 5.8995545e-01 9.0088099e-01 4.4530898e-01]
 [0.0000000e+00 5.7090628e-01 1.4106162e+00 1.8950461e-03 2.7044770e-01]
 [7.4149209e-01 4.8234680e-01 1.2242768e+00 1.1730905e+00 1.2739047e+00]
 [5.5356252e-01 4.7456071e-01 2.1302395e-01 1.1839507e+00 6.7052907e-01]
 [4.2669445e-01 9.4983858e-01 4.8905596e-01 7.2570872e-01 7.9190499e-01]
 [2.9646337e-01 3.6512226e-02 3.1079498e-01 2.3691502e-01 2.5772560e-01]
 [3.1486410e-01 1.0816